# Digital Speech Processing
### Prof. Dr. Rodrigo Guido

### Student: Álvaro Leandro Cavalcante Carneiro

# Final Term Exam
The final term exam contains 10 different exercises that'll be documented in this notebook. Some of them will be solved using Python programming language, and others will be using the paper, but all the content will be present here.

## Importing all the necessary libraries

In [115]:
import math
import numpy as np
from scipy.spatial import distance
import random

# Exercise number 1
Creating a band-stop filter with normalized coefficients.

The idea of this exercise is very similar to the short test 6. First of all, we need to obtain a low-pass filter and a high-pass filter, taking care to get normalized values. After that, we just sum up our obtained low-pass filter with the high-pass one, to get the band-stop filter. We can verify that the obtained filter is symmetric, giving a linear phase response. Besides that, we can verify that the band-stop filter has a gain of 0 dB. In the end, we use the difference equation method to filter a random signal with the band-stop coefficients. 

In [33]:
filter_order = 8
samples_second = 22050
cutoff_freq_a = 2000
cutoff_freq_b = 4000

In [34]:
def get_low_pass_filter(filter_order, samples_second, cutoff_frequency, high_pass=False):
    frequency = samples_second / 2
    divisor = frequency / 1000
    dividend = cutoff_frequency / 1000 
    
    if high_pass:
        dividend = divisor - dividend

    final_filter = []

    for n in range(filter_order + 1):
        first_part = math.sin( ((dividend * math.pi) / divisor) * (n - (filter_order/2)) )
        second_part = (math.pi * (n - (filter_order/2)))
        if second_part == 0:
            value = 1
        else:
            value = first_part / second_part
        final_filter.append(value)
    
    return np.array(final_filter)

In [35]:
def normalized_coef(coef):
    print('Sum befone normalization', sum(coef))
    normalized = [x/sum(coef) for x in coef]
    print('Sum after normalization', sum(normalized))
    return normalized

In [36]:
low_pass_filter = normalized_coef(
    get_low_pass_filter(filter_order, samples_second, cutoff_freq_a, False))
low_pass_filter

Sum befone normalization 1.9636720577105782
Sum after normalization 1.0


[0.030763701937615313,
 0.05351260046295566,
 0.07363789309032391,
 0.08746080333505772,
 0.5092500023480948,
 0.08746080333505772,
 0.07363789309032391,
 0.05351260046295566,
 0.030763701937615313]

In [37]:
low_pass_b = normalized_coef(
    get_low_pass_filter(filter_order, samples_second, cutoff_freq_b, True))
low_pass_b

Sum befone normalization 1.4358538558926381
Sum after normalization 0.9999999999999998


[0.05477292274475763,
 -0.020267164783418494,
 -0.08414480573871011,
 0.2014141933132226,
 0.6964497089282965,
 0.2014141933132226,
 -0.08414480573871011,
 -0.020267164783418494,
 0.05477292274475763]

In [38]:
def get_high_pass_filter(filter_value):      
    reversed_filter = np.array(filter_value)[::-1]
    inverse = []
    for i, value in enumerate(reversed_filter):
        if i % 2 != 0:
            inverse.append(-value)
        else:
            inverse.append(value)
       
    return np.array(inverse)

In [39]:
high_pass = get_high_pass_filter(low_pass_b)
high_pass

array([ 0.05477292,  0.02026716, -0.08414481, -0.20141419,  0.69644971,
       -0.20141419, -0.08414481,  0.02026716,  0.05477292])

In [50]:
band_stop = normalized_coef(np.array(low_pass_filter) + np.array(high_pass))
list(band_stop)

Sum befone normalization 1.2754118858807832
Sum after normalization 1.0


[0.06706588328781524,
 0.057847794946197156,
 -0.008238054517682544,
 -0.08934634469042142,
 0.9453414419481831,
 -0.08934634469042142,
 -0.008238054517682544,
 0.057847794946197156,
 0.06706588328781524]

In [51]:
print(20 * np.log(sum(band_stop)), 'dB gain')

0.0 dB gain


### Difference equation
Considering a random input filter x, let's apply the difference equation with our filter coefficients.

In [101]:
def get_random_filter(max_len):
    x = []
    for _ in range(max_len):
        x.append(random.random())

    return x

In [110]:
x = get_random_filter(9)
print(x[0:30])

[0.336875531114773, 0.788330490289538, 0.10686149796621658, 0.9209008312441466, 0.052816485448852535, 0.6001518326235036, 0.045153933877495, 0.531487963025663, 0.9712112276949454]


In the code bellow we apply the difference equation in the random signal. I'm showing just the 30 first resulting values.

In [111]:
def apply_diff_equation(x, freq_filter):
    y = []
    order = len(freq_filter)

    for n in range(0, len(x), order):
        input_sample = x[n: n + order][::-1]
        y[n: n + order] = np.multiply(freq_filter, input_sample)
        
    return y

In [113]:
y = apply_diff_equation(x, band_stop)
y

[0.06513513884440496,
 0.03074540670148057,
 -0.00037198056897063656,
 -0.05362137250416765,
 0.04992961251285349,
 -0.08227912309403512,
 -0.0008803308460869145,
 0.045603180552104265,
 0.022592855052264136]

# Exercise number 2
Applying a window to the filter.

In [88]:
upper_bound_freq = 0.4
lower_bound_freq = 0.1

passband_upper = 1.02
passband_lower = 0.98

stopband_upper = 0.06
stopband_lower = 0

cutoff = upper_bound_freq - lower_bound_freq

In [89]:
passband_range = passband_upper - passband_lower
stopband_range = stopband_upper - stopband_lower

min_frequency = min([passband_range, stopband_range])

In [90]:
dB_to_be_filtered = 20* math.log(min_frequency, 10)

windows = {'rectangular': -21, 'barlett': -25, 'hanning': -44, 'hamming': -53, 'blackman': -74}
used_window = ''

for window in windows:
    if windows[window] < dB_to_be_filtered:
        used_window = window
        print('Window to be used:', window)
        break

Window to be used: hanning


In [91]:
order = int(np.ceil(3.1 / ((cutoff * math.pi) / (2 * math.pi))))
print('Filter order is:', order, 'Which will generate:', order+1, 'coefficients.')

Filter order is: 21 Which will generate: 22 coefficients.


In [92]:
def get_low_pass_filter(order, cutoff):
    final_filter = []
    for n in range(order + 1):
        first_part = math.sin( (cutoff * math.pi)  * (n - (order/2)) )
        second_part = (math.pi * (n - (order/2)))
        if second_part == 0:
            print('Zero division')
            value = 1
        else:
            value = first_part / second_part
        final_filter.append(value)
    
    return final_filter

In [93]:
freq_filter = get_low_pass_filter(order, cutoff)
print(sum(freq_filter))
freq_filter

1.0412224393462681


[-0.013762825171487384,
 0.015211543610591289,
 0.03698717215057592,
 0.030010543871903557,
 -0.007660713348027363,
 -0.05156657914607532,
 -0.06302581895631432,
 -0.014227039074908047,
 0.09003163161571058,
 0.20959397551993025,
 0.2890193286012348,
 0.2890193286012348,
 0.20959397551993025,
 0.09003163161571058,
 -0.014227039074908047,
 -0.06302581895631432,
 -0.05156657914607532,
 -0.007660713348027363,
 0.030010543871903557,
 0.03698717215057592,
 0.015211543610591289,
 -0.013762825171487384]

In [94]:
def get_window_value(order):
    window = []
    for n in range(order + 1):
        value = 0.5 - (0.5* math.cos(2*math.pi*(n/ order)))
        window.append(value)
    return window

In [95]:
han_window = get_window_value(order)
print(len(han_window))
han_window

22


[0.0,
 0.022213597106929606,
 0.08688061284200255,
 0.1882550990706332,
 0.31732948781680237,
 0.4626349532067878,
 0.6112604669781572,
 0.7499999999999999,
 0.8665259359149131,
 0.9504844339512095,
 0.9944154131125642,
 0.9944154131125642,
 0.9504844339512095,
 0.8665259359149131,
 0.7500000000000002,
 0.6112604669781573,
 0.46263495320678827,
 0.3173294878168027,
 0.1882550990706333,
 0.08688061284200255,
 0.022213597106929717,
 0.0]

In [96]:
window_filter = np.array(han_window) * np.array(freq_filter)
window_filter

array([-0.        ,  0.0003379 ,  0.00321347,  0.00564964, -0.00243097,
       -0.0238565 , -0.03852519, -0.01067028,  0.07801474,  0.19921581,
        0.28740528,  0.28740528,  0.19921581,  0.07801474, -0.01067028,
       -0.03852519, -0.0238565 , -0.00243097,  0.00564964,  0.00321347,
        0.0003379 , -0.        ])

In [97]:
filter_normalized = normalized_coef(window_filter)
filter_normalized

Sum befone normalization 0.9967077925322666
Sum after normalization 1.0000000000000002


[-0.0,
 0.0003390192227570301,
 0.0032240825323241917,
 0.00566829912648234,
 -0.002438999936846753,
 -0.02393530190996913,
 -0.03865244338969116,
 -0.01070552411261057,
 0.0782724329359798,
 0.1998738373214789,
 0.2883545982100954,
 0.2883545982100954,
 0.1998738373214789,
 0.0782724329359798,
 -0.010705524112610575,
 -0.03865244338969117,
 -0.023935301909969155,
 -0.0024389999368467554,
 0.005668299126482344,
 0.0032240825323241917,
 0.00033901922275703176,
 -0.0]

In [105]:
x = get_random_filter(22)
print(x[0:30])

[0.43234330523796427, 0.5199255822612687, 0.04324021077327356, 0.2239541077115943, 0.9972540725574214, 0.9931029371772746, 0.5828811564428854, 0.5842405992181255, 0.6352666259200075, 0.8683377217945182, 0.4640258371702567, 0.05950684927532679, 0.23991152462091347, 0.37061792031858476, 0.7036962613014697, 0.5942667024379145, 0.6460506136612234, 0.41642702836107937, 0.11860724192734695, 0.4829591430603736, 0.9508635057824045, 0.9181074019303173]


In [106]:
y = apply_diff_equation(x, filter_normalized)
y[0:30]

[-0.0,
 0.0003223610066783756,
 0.0015571001369672109,
 0.0006723013258112603,
 -0.0010156654958739535,
 -0.01546341648710221,
 -0.022969860074359933,
 -0.0075334372933167915,
 0.02900916631300873,
 0.04795203704362844,
 0.017159073613535563,
 0.13380398383633252,
 0.17355799254606114,
 0.04972386437378995,
 -0.006254601822495694,
 -0.022529780902326352,
 -0.023770218629015197,
 -0.0024323026199877204,
 0.0012694388731137626,
 0.0001394100082481276,
 0.00017626476678971251,
 -0.0]

![diagram](assets/2_part1.jpeg)


# Exercise number 3
What's a window and which is the function of it?

An FIR signal is just the result of a subsample of the SINC function. As this function is infinite, the subsample must be done with a limited amount of coefficients and in a limited part of the function, enabling the computational representation. This part is what we call a window, which is the part of the SINC that we'll considered, all the other parts may be replaced by 0. The window that was chosen to sample the signal may cause the Gibbs effect, which's a fluctuation caused by the SINC truncation, disturbing the frequency response of the signal. A window that truncates the SINC function in a smoother manner may reduce the Gibbs effect and is desirable. For example, a rectangular window may produce a abrupt "cutoff" in the function, generating a higher Gibbs effect, while a triangular one will be smoother in the truncation.

# Exercise number 4
Finding poles and zeros. This exercise was done "by hand" as showed by the images bellow

![part one](assets/exer_4_1.jpeg)

![part two](assets/4_2.jpeg)

# Exercise number 5
Finding the length of a subsampled signal. The first step is to define the signal, samples per second and ms that covers the subsample.

In [127]:
signal = [-1, 2, -3, 3, 2, 1, -1, -1, -4, 5, 5, 4]
ss = 16000
ms = 0.125
overlap = 0.5

We now get the len of the window that will be used to recover the signal values.

In [134]:
def get_window_size(ss, ms):
    window_size = (ss * ms) / 1000
    return int(window_size)

In [168]:
w_size = get_window_size(ss, ms)
print('The rectangular window has a size of: ', w_size)

The rectangular window has a size of:  2


In [150]:
window_step = int(w_size * overlap)

In [170]:
final_signal = []
for i in range(0, len(signal), window_step):
    cutoff = signal[i:i+window_step+1]
    final_signal.extend(cutoff)
    
print('The f[n] signal has a len of ', i)
print(final_signal)

The f[n] signal has a len of  11
[-1, 2, 2, -3, -3, 3, 3, 2, 2, 1, 1, -1, -1, -1, -1, -4, -4, 5, 5, 5, 5, 4, 4]


In the end, we create a function to get the entropy based on the probability of each value of the signal to appear.

In [171]:
def get_entropy(signal, base):
    total_len = len(signal)
    values, counts = np.unique(signal, return_counts=True)
    probability = list(map(lambda x: x/total_len, counts))

    if(base == 2):
        entropy = np.array(
            list(map(lambda value: value * np.log(value), probability)))
    elif (base == 10):
        entropy = np.array(
            list(map(lambda value: -1 * (value * np.log10(value)), probability)))

    return np.sum(entropy)

In [174]:
print('The entropy of the signal is:', get_entropy(final_signal, 10))

The entropy of the signal is: 0.869483057367871


# Exercise number 6
Bark scale concept.

The cochlea is an organ present in the human auditory system that responds to stimuli generated by the received signals, entering into vibration and producing electrical signals that are transmitted to the brain. That said, the Bark scale is the result of an experimental mapping containing 25 frequency bands that stimulate specific regions of the cochlea, producing energy that will be interpreted by our brain.

Thus, digital signal filtering and handcrafted feature extraction were bioinspired to carry out a similar process. This process occurs through the creation of 25 filters with the same frequency ranges as the Bark scale. Once this is done, the signal of interest is filtered 25 times and has the energy extracted, generating a feature vector with, theoretically, the energy produced by each band of the Bark scale. In the same way that energy is interpreted by the brain, the vector is used in a classifier in order to find some pattern of interest.

# Exercise number 7
Resolving a order 4 LPC. The first step is to define the order and the signal

In [69]:
order = 4
signal = np.array([2, 5, 2, 3, 5, 8, 4, 8, 10, 6])

x_matrix = np.array([])
y = np.array([])

After that, we need to get the matrix of values and Y results for the equation

In [70]:
for s in range(0, len(signal) - order):
    x_matrix = np.append(x_matrix, signal[s: s+order])
    y = np.append(y, signal[order+s])
    
x_matrix = x_matrix.reshape(len(signal)-order, order)

print('Matrix of values \n', x_matrix)
print('Y results \n', y)

Matrix of values 
 [[ 2.  5.  2.  3.]
 [ 5.  2.  3.  5.]
 [ 2.  3.  5.  8.]
 [ 3.  5.  8.  4.]
 [ 5.  8.  4.  8.]
 [ 8.  4.  8. 10.]]
Y results 
 [ 5.  8.  4.  8. 10.  6.]


In the end, we calculate the dot product and use a numpy function to get the coefficients based on linear algebra.

In [72]:
x_result = np.dot(x_matrix.T, x_matrix)
y_result = np.dot(x_matrix.T, y)

print('X result \n', x_result)
print('Y result \n', y_result)

X result 
 [[131. 113. 137. 179.]
 [113. 143. 135. 173.]
 [137. 135. 182. 205.]
 [179. 173. 205. 278.]]
Y result 
 [180. 197. 206. 259.]


In [73]:
np.linalg.solve(x_result, y_result)

array([ 0.69745441,  0.9515517 ,  0.14442733, -0.21608053])

# Exercise number 8
The difference between learned and handcrafted features

We can interpret a feature as an input signal that's passed through a classifier. When working with unstructured data, like images and speech signals, there's a huge amount of data, but most of this data is not really useful for the model, because it's not discriminative. Based on this, a common practice in this area is to use some method (handcrafted or learned) to extract some features of this amount of data that'll be representative and will aggregate predictive power.

The handcrafted features receive this name because it needs the direct intervention of the engineer that's designing the solution, while the learned-based features are automatically created by a complex model based on a training loop, requiring less knowledge about the signal. In the handcrafted version, the engineer needs to know what's the specific representation of the signal that's useful to extract for the problem that needs to be resolved and also which formula must be used to get the feature. For example, if someone wants to create a system that recognizes voiced signals, the engineer should know that energy and zero-crossing rate are relevant pieces of information to classify the signal between voiced and unvoiced. Moreover, these calculations need to be done to extract the desired features (energy and zero-crossing) and pass them to a classifier. 

The main advantage of the handcrafted version is that all the physics properties of the signal used in the classifier are known, and the final result is simpler to be interpreted and explained. Furthermore, the handcrafted version is usually cheaper computationally because only the required calculations are done to extracted the desired properties. The main disadvantage of this process is that it needs specialized knowledge and usually takes longer to create the solution.

The learned features, on the other hand, are based on complex models like neural networks that, based on a training loop, can automatically learn the best representations of the signal. The advantage of this approach is that it's faster, easier, and reaches good results, sometimes better than those reached with handcrafted features. The disadvantages are the computational cost and the resulting black box, because the physics properties explored by the model are not known and, until this moment, there is no way to discover how to model reach that solution.

# Exercise number 9
Hidden Markov Model modeling.

![part one](assets/9_p1.jpeg)
![part two](assets/9_p2.jpeg)

# Exercise number 10
Estimate the most similar signals using euclidian distance. The first step is to define the template signal and the signals that will be compared.

In [116]:
c_a = [0.2, 1, 0.7]
c_b = [0.3, 0.9, 0.8]
t = [0.2, 0.9, 0.9]

After that, we use the method above to get the Euclidian distance.

In [117]:
def get_euclidian_distance(template, signals_to_compare):
    dist = []
    for i in range(len(signals_to_compare)):
        subtracted = np.array(template) - np.array(signals_to_compare[i])
        dist.append(math.sqrt(sum(subtracted**2)))
        
    return dist

In [121]:
signals_to_compare = [c_a, c_b]
dist = get_euclidian_distance(t, signals_to_compare)
dist

[0.22360679774997902, 0.14142135623730948]

In [122]:
most_similar = dist.index(min(dist))

In [124]:
print('The signal that best matches with the template is: ', signals_to_compare[most_similar])

The signal that best matches with the template is:  [0.3, 0.9, 0.8]


If the signals had different dimensions we may use another technique to find the best match, like Dynamic Time Warping or KNN.